In [1]:
%pip install transformers
%pip install torch
%pip install spacy
%python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\Nont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\Nont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\Nont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [2]:
import transformers
import json
import random
# import tensorflow
import torch
import spacy
# import nlp

C:\Users\Nont\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
loaded_classifier = transformers.pipeline("zero-shot-classification", model="./simple_trained_wsd_pipeline")

In [4]:
semcor_samples = "semcor_samples_4-samples.json"

In [5]:
with open(semcor_samples, 'r') as f:
    full_samples = json.load(f)

In [6]:
# print(full_samples)

In [7]:
uniq_sent_idx = list(set(full_samples['index']))

In [8]:
random.Random(4).shuffle(uniq_sent_idx)
train_idx = uniq_sent_idx[:24138]
val_idx = uniq_sent_idx[24138:24138+5173]
test_idx = uniq_sent_idx[24138+5173:24138+5173+5173]

In [9]:
len(train_idx), len(val_idx), len(test_idx)

(24138, 5173, 5173)

# Re-test the model

In [10]:
test_samples = {
    'sentence': [],
    'hypothesis': [],
    'label': []
}

test_idx = set(test_idx)

for i in range(len(full_samples['sentence'])):
    curr_idx = full_samples['index'][i]
    
    if curr_idx in test_idx:
        test_samples['sentence'].append(full_samples['sentence'][i])
        test_samples['hypothesis'].append(full_samples['hypothesis'][i])
        test_samples['label'].append(full_samples['label'][i])

In [11]:
answers_trained = []
for i in range(len(test_samples['sentence'])):
  if i % 4 == 0:
    sequence_to_classify = test_samples['sentence'][i]

    candidate_labels = test_samples['hypothesis'][i:i+4]

    sen = sequence_to_classify.split(" [")

    try:
      sen = sen[1].split("] ")
    except(IndexError):
      break

    wword = sen[0]

    hypothesis_template = "{}"

    output = loaded_classifier(sequence_to_classify, candidate_labels, hypothesis_template=hypothesis_template)

    label_list = []
    for label, score in zip(output['labels'], output['scores']):
        # print(f"{score:.5f} || {label}")
        label_list.append(label)
    answers_trained.append(label_list[0])

In [12]:
correct_ref = []
for j in range(len(test_samples['hypothesis'])):
  if j % 4 == 0:
    # print(test_samples['hypothesis'][j])
    correct_ref.append(test_samples['hypothesis'][j])

  if j > 466:
    break

In [13]:
# answers = []
# for i in range(len(test_samples['sentence'])):
#   if i % 4 == 0:
#     sequence_to_classify = test_samples['sentence'][i]

#     candidate_labels = test_samples['hypothesis'][i:i+4]

#     sen = sequence_to_classify.split(" [")

#     try:
#       sen = sen[1].split("] ")
#     except(IndexError):
#       break

#     wword = sen[0]

#     hypothesis_template = "{}"

#     output = loaded_classifier(sequence_to_classify, candidate_labels, hypothesis_template=hypothesis_template)

#     label_list = []
#     for label, score in zip(output['labels'], output['scores']):
#         # print(f"{score:.5f} || {label}")
#         label_list.append(label)
#     answers.append(label_list[0])

In [14]:
nlp = spacy.load("en_core_web_sm")

In [15]:
correct_trained = 0
wrong_trained = 0
collector = {}

for n in range(len(correct_ref)):
  if correct_ref[n] == answers_trained[n]:
    k = correct_ref[n].split(" [")
    k = k[1].split("] ")
    k = k[0]
    ex0 = nlp(k)

    state = "correct"

    # collector[f"{n}, {state}"] = []
    
    for word0 in ex0:
        sentence_question_tag0 = k.replace(word0.text, f"[{word0.text}]")
        # print(word0.lemma_)
        original0 = sentence_question_tag0.replace("[", "").replace("]", "")
        # print(original0)
        # print(word0.pos_)
        collector[f"{n}, {state}"] = (f"{original0} = {word0.pos_}")
    
    print("correct part")
    # print(correct_ref[n])
    # print(answers[n])
    correct_trained += 1
  else:
    # for label, score in zip(output['labels'], output['scores']):
    #     print(f"{score:.5f} || {label}")
        # correct_def[key4].append(f"{score:.5f}, {label}")
    # print(word.pos_)

    # print("❌", answers_trained[n])
    # print("🟢", correct_ref[n])
    state = "wrong"

    # collector[f"{n}, {state}"] = []

    m = correct_ref[n].split(" [")
    m = m[1].split("] ")
    m = m[0]
    ex1 = nlp(m)

    for word1 in ex1:
        sentence_question_tag1 = m.replace(word1.text, f"[{word1.text}]")
        # print(word1.lemma_)
        original1 = sentence_question_tag1.replace("[", "").replace("]", "")
        # print(original1)
        # print(word1.pos_)
        collector[f"{n}, {state}"] = (f"{original0} = {word0.pos_}")
    
    print("wrong part")
    wrong_trained += 1
    print(collector)

correct part
correct part
correct part
correct part
wrong part
{'0, correct': 'take = VERB', '1, correct': 'steps = NOUN', '2, correct': 'said = VERB', '3, correct': 'realize = VERB', '4, wrong': 'realize = VERB'}
correct part
correct part
correct part
correct part
correct part
wrong part
{'0, correct': 'take = VERB', '1, correct': 'steps = NOUN', '2, correct': 'said = VERB', '3, correct': 'realize = VERB', '4, wrong': 'realize = VERB', '5, correct': 'program = NOUN', '6, correct': 'matters = NOUN', '7, correct': 'filed = VERB', '8, correct': 'suit = NOUN', '9, correct': 'charged = VERB', '10, wrong': 'charged = VERB'}
wrong part
{'0, correct': 'take = VERB', '1, correct': 'steps = NOUN', '2, correct': 'said = VERB', '3, correct': 'realize = VERB', '4, wrong': 'realize = VERB', '5, correct': 'program = NOUN', '6, correct': 'matters = NOUN', '7, correct': 'filed = VERB', '8, correct': 'suit = NOUN', '9, correct': 'charged = VERB', '10, wrong': 'charged = VERB', '11, wrong': 'charged = V

In [16]:
print(collector)

{'0, correct': 'take = VERB', '1, correct': 'steps = NOUN', '2, correct': 'said = VERB', '3, correct': 'realize = VERB', '4, wrong': 'realize = VERB', '5, correct': 'program = NOUN', '6, correct': 'matters = NOUN', '7, correct': 'filed = VERB', '8, correct': 'suit = NOUN', '9, correct': 'charged = VERB', '10, wrong': 'charged = VERB', '11, wrong': 'charged = VERB', '12, correct': 'became = VERB', '13, correct': 'primary = NOUN', '14, correct': 'announced = VERB', '15, wrong': 'announced = VERB', '16, correct': 'added = VERB', '17, wrong': 'added = VERB', '18, wrong': 'added = VERB', '19, correct': 'rally = NOUN', '20, correct': 'held = VERB', '21, correct': 'be = AUX', '22, correct': 'source = NOUN', '23, correct': 'told = VERB', '24, correct': 'however = ADV', '25, wrong': 'however = ADV', '26, wrong': 'however = ADV', '27, correct': 'issue = NOUN', '28, correct': 'bonds = NOUN', '29, correct': 'said = VERB', '30, correct': 'night = NOUN', '31, wrong': 'night = NOUN', '32, correct': '

In [17]:
for info in collector.items():
    # print(info)
    # print(type(info))
    print(info[0])
    print(info[1])

0, correct
take = VERB
1, correct
steps = NOUN
2, correct
said = VERB
3, correct
realize = VERB
4, wrong
realize = VERB
5, correct
program = NOUN
6, correct
matters = NOUN
7, correct
filed = VERB
8, correct
suit = NOUN
9, correct
charged = VERB
10, wrong
charged = VERB
11, wrong
charged = VERB
12, correct
became = VERB
13, correct
primary = NOUN
14, correct
announced = VERB
15, wrong
announced = VERB
16, correct
added = VERB
17, wrong
added = VERB
18, wrong
added = VERB
19, correct
rally = NOUN
20, correct
held = VERB
21, correct
be = AUX
22, correct
source = NOUN
23, correct
told = VERB
24, correct
however = ADV
25, wrong
however = ADV
26, wrong
however = ADV
27, correct
issue = NOUN
28, correct
bonds = NOUN
29, correct
said = VERB
30, correct
night = NOUN
31, wrong
night = NOUN
32, correct
done = VERB
33, correct
vote = VERB
34, correct
increase = NOUN
35, wrong
increase = NOUN
36, wrong
increase = NOUN
37, correct
given = VERB
38, correct
action = NOUN
39, correct
sought = VERB
40, 

# Start Error Analysis

## Count the number of correct and wrong predictions

In [23]:
correct = 0
wrong = 0

collector_correct = []
collector_wrong = []

for count in collector.items():
    # print(count)
    holder = count[0]
    number, prediction = holder.split(", ")
    # print(prediction)
    # print(number)
    # print(type(number))
    # print(number, prediction)
    if prediction == "correct":
        correct += 1
        collector_correct.append(count)

    if prediction == "wrong":
        wrong += 1
        collector_wrong.append(count)

print("correct: ", correct, "words")
print("wrong  :", wrong, "words")

correct:  89 words
wrong  : 28 words


## Find the types of part of speech of the correct and wrong vocab predictions

In [69]:
noun_correct = 0
verb_correct = 0
adj_correct = 0
adv_correct = 0
aux_correct = 0

noun_wrong = 0
verb_wrong = 0
adj_wrong = 0
adv_wrong = 0
aux_wrong = 0

for pos_type0 in collector_correct:
    holder = pos_type0[1]
    user_vocab, ppos = holder.split(" = ")
    if ppos == "NOUN":
        noun_correct += 1
    if ppos == "VERB":
        verb_correct += 1
    if ppos == "ADJ":
        adj_correct += 1
    if ppos == "ADV":
        adv_correct += 1
    # if ppos != "NOUN" and ppos != "VERB" and ppos != "ADJ" and ppos != "ADV":
    #     print("EXCEPTION:", ppos)
    if ppos == "AUX":
        aux_correct += 1

print(">> Correct Predictions Report")
print("noun :", noun_correct, "words")
print("verb :", verb_correct, "words")
print("adj  :", adj_correct, "words")
print("adv  :", adv_correct, "words")
print("aux  :", aux_correct, "words")
print("total:", noun_correct+verb_correct+adj_correct+adv_correct+aux_correct, "words")

for pos_type1 in collector_wrong:
    # print(pos_type)
    holder = pos_type1[1]
    user_vocab, ppos = holder.split(" = ")
    if ppos == "NOUN":
        noun_wrong += 1
    if ppos == "VERB":
        verb_wrong += 1
    if ppos == "ADJ":
        adj_wrong += 1
    if ppos == "ADV":
        adv_wrong += 1
    if ppos == "AUX":
        aux_wrong += 1

print("")
print(">> Wrong Predictions Report")
print("noun :", noun_wrong, "words")
print("verb :", verb_wrong, "words")
print("adj  :", adj_wrong, "words")
print("adv  :", adv_wrong, "words")
print("aux  :", aux_wrong, "words")
print("total:", noun_wrong+verb_wrong+adj_wrong+adv_wrong+aux_wrong, "words")

>> Correct Predictions Report
noun : 40 words
verb : 42 words
adj  : 1 words
adv  : 1 words
aux  : 5 words
total: 89 words

>> Wrong Predictions Report
noun : 9 words
verb : 17 words
adj  : 0 words
adv  : 2 words
aux  : 0 words
total: 28 words


## Find the average vocab length of the correct and wrong vocab prediction

In [63]:
length_total_correct = 0

# print(collector_correct)
amount_correct = len(collector_correct)
for set in collector_correct:
    # print(set)
    vocabulary_correct, pos_correct = set[1].split(" = ")
    print(vocabulary_correct)
    length_total_correct += len(vocabulary_correct)

print("total correct length:", length_total_correct)
print("amount correct:", amount_correct)
print("An average correct vocab length:", round(length_total_correct/amount_correct), "characters")

take
steps
said
realize
program
matters
filed
suit
charged
became
primary
announced
added
rally
held
be
source
told
however
issue
bonds
said
night
done
vote
increase
given
action
sought
rejected
vote
sessions
received
election
got
election
campaign
received
was
trouble
approval
property
act
seemed
witnesses
passage
bill
said
failure
making
banks
bill
is
said
hear
special
adopted
vote
kept
bill
votes
representatives
joined
holds
degree
studied
report
culminating
year
study
program
designed
reduce
is
dollars
said
president
forced
people
said
cases
reported
community
is
deal
employment
said
causes
family
total correct length: 520
amount correct: 89
An average correct vocab length: 6 characters


In [62]:
length_total_wrong = 0

# print(collector_wrong)
amount_wrong = len(collector_wrong)
for set in collector_wrong:
    # print(set)
    vocabulary_wrong, pos_wrong = set[1].split(" = ")
    print(vocabulary_wrong)
    length_total_wrong += len(vocabulary_wrong)

print("total wrong length:", length_total_wrong)
print("amount wrong:", amount_wrong)
print("An average wrong vocab length:", round(length_total_wrong/amount_wrong), "characters")

realize
charged
charged
announced
added
added
however
however
night
increase
increase
rejected
vote
received
seemed
seemed
seemed
witnesses
making
hear
hear
bill
year
program
said
employment
said
family
total wrong length: 175
amount wrong: 28
An average wrong vocab length: 6 characters


In [68]:
print("total correct vocab length     :", length_total_correct, "characters")
print("number of correct vocab        :", amount_correct, "words")
print("An average correct vocab length:", round(length_total_correct/amount_correct), "characters")
print("")
print("total wrong vocab length       :", length_total_wrong, "characters")
print("number of wrong vocab          :", amount_wrong, "words")
print("An average wrong vocab length  :", round(length_total_wrong/amount_wrong), "characters")

total correct vocab length     : 520 characters
number of correct vocab        : 89 words
An average correct vocab length: 6 characters

total wrong vocab length       : 175 characters
number of wrong vocab          : 28 words
An average wrong vocab length  : 6 characters


In [29]:
print(correct_trained, wrong_trained)

89 28


In [21]:
total_trained = correct_trained + wrong_trained
cpercent = round(((correct_trained / total_trained) * 100), 2)
wpercent = round(((wrong_trained / total_trained) * 100), 2)
tpercent = round(((total_trained / total_trained) * 100), 2)
print(f"Correct: {correct_trained}  ({cpercent}%)")
print(f"Wrong  : {wrong_trained}  ({wpercent}%)")
print(f"Total  : {total_trained} ({tpercent}%)")

Correct: 89  (76.07%)
Wrong  : 28  (23.93%)
Total  : 117 (100.0%)


In [22]:
# accuracy calculation

accuracy_trained = (correct_trained / (correct_trained + wrong_trained)) * 100
# print(accuracy_trained)
accuracy_trained = str(round(accuracy_trained, 2))

print("")
print("----------- TEST RESULT -----------")
print("Model    : ./simple_trained_wsd_pipeline")
print("State    : After fine-tuning")
print("Accuracy : " + accuracy_trained + "%")


----------- TEST RESULT -----------
Model    : ./simple_trained_wsd_pipeline
State    : After fine-tuning
Accuracy : 76.07%
